In [3]:
# read ontology "legal_ontology.owl"
# Joint_Circulars is subclass of Legal_Normative_Document
# Legal_Normative_Document is subclass of Legal_Document
# Legal_Document is subclass of Thing
# write SPAQRL query to get constraints of Joint_Circulars

from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery

g = Graph()
g.parse("legal_onto.owl")

q = prepareQuery('''
	SELECT ?x ?y
	WHERE {
		?x rdfs:subClassOf ?y .
		?x rdfs:label ?label .
		FILTER regex(?label, "Joint_Circulars", "i")
	}
	''',
	initNs = { "rdfs": "http://www.w3.org/2000/01/rdf-schema#" }
)

for r in g.query(q):
	print(r)

In [9]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL

# Create a graph
g = Graph()

# Load your ontology (replace 'ontology.owl' with the path or URL of your ontology)
g.parse('legal_onto.owl', format='xml')

# Define relevant namespaces (update the URL based on your ontology namespace)
EX = Namespace("http://www.semanticweb.org/ontologies/2024/8/#")

# SPARQL query to extract constraints of Joint_Circulars
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?property ?constraintType ?constraintValue
WHERE {
    # Specify the class Joint_Circulars (change IRI to match your ontology)
    ?property rdfs:domain <http://example.com/ontology#Joint_Circulars> .

    # Fetch range constraints (if any)
    OPTIONAL { ?property rdfs:range ?constraintValue .
               BIND("range" AS ?constraintType) }
    
    # Fetch min/max cardinality constraints
    OPTIONAL { ?property owl:minCardinality ?constraintValue .
               BIND("minCardinality" AS ?constraintType) }
    OPTIONAL { ?property owl:maxCardinality ?constraintValue .
               BIND("maxCardinality" AS ?constraintType) }
    
    # Fetch other kinds of constraints, e.g., allValuesFrom
    OPTIONAL { ?restriction rdf:type owl:Restriction ;
                             owl:onProperty ?property ;
                             owl:allValuesFrom ?constraintValue .
               BIND("allValuesFrom" AS ?constraintType) }
}
"""

# Run the query
results = g.query(query)
print(list(results))
# Display results
for row in results:
    print(f"Property: {row.property}, Constraint Type: {row.constraintType}, Constraint Value: {row.constraintValue}")

[]


In [10]:
import rdflib

# Load the ontology from an OWL file
g = rdflib.Graph()
g.parse("legal_onto.owl")

# Define the SPARQL query to extract constraints on Joint_Circulars
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?property ?restriction ?value
WHERE {
  ?jointCirculars rdf:type owl:Class ;
                  owl:equivalentClass ?equivalentClass .
  
  ?equivalentClass owl:intersectionOf ?intersection .
  ?intersection rdf:rest*/rdf:first ?restriction .
  
  ?restriction owl:onProperty ?property ;
               ?constraintType ?value .
  FILTER (?constraintType IN (owl:minQualifiedCardinality, owl:someValuesFrom, owl:onClass))
}
"""

# Execute the query
results = g.query(query)

# Print the results
for row in results:
    print(f"Property: {row.property}, Restriction: {row.restriction}, Value: {row.value}")


Property: http://www.semanticweb.org/ontologies/2024/8/legal_ontology#canBePromulgatedBy, Restriction: N19a82e17fde24424913eb29855e7ce6d, Value: N20119ebabaeb405f9243151cf3f1dd98
Property: http://www.semanticweb.org/ontologies/2024/8/legal_ontology#canBePromulgatedBy, Restriction: Ncd3ee42b593f467a998d77414aa404c1, Value: Nc4f83f8b967247f3bad6290b07614f4f
Property: http://www.semanticweb.org/ontologies/2024/8/legal_ontology#canBePromulgatedBy, Restriction: Ncd3ee42b593f467a998d77414aa404c1, Value: 2
Property: http://www.semanticweb.org/ontologies/2024/8/legal_ontology#canBePromulgatedBy, Restriction: N53eeb6516bac4140bae5c11343cc7579, Value: Nf5de7086a1a845d3a76d661a32f71de1


In [11]:
import rdflib

# Load the ontology from an OWL file
g = rdflib.Graph()
g.parse("legal_onto.owl")

# Define the SPARQL query to extract human-readable constraints on Joint_Circulars
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?property ?restrictionLabel ?valueLabel ?cardinality
WHERE {
  ?jointCirculars rdf:type owl:Class ;
                  owl:equivalentClass ?equivalentClass .
  
  ?equivalentClass owl:intersectionOf ?intersection .
  ?intersection rdf:rest*/rdf:first ?restriction .
  
  ?restriction owl:onProperty ?property ;
               owl:onClass|owl:someValuesFrom ?value .
  
  OPTIONAL { ?restriction rdfs:label ?restrictionLabel . }
  OPTIONAL { ?value rdfs:label ?valueLabel . }
  
  OPTIONAL {
    ?restriction owl:minQualifiedCardinality ?cardinality .
  }
  
  # In case of blank nodes, fall back to showing the URI or value
  BIND(COALESCE(?restrictionLabel, STR(?restriction)) AS ?restrictionReadable)
  BIND(COALESCE(?valueLabel, STR(?value)) AS ?valueReadable)
}
"""

# Execute the query
results = g.query(query)

# Print the results in a human-readable format
for row in results:
    property = row.property.split('#')[-1]  # Extract readable property name
    restriction = row.restrictionReadable
    value = row.valueReadable
    cardinality = row.cardinality if row.cardinality else "N/A"
    print(f"Property: {property}, Restriction: {restriction}, Value: {value}, Cardinality: {cardinality}")


AttributeError: restrictionReadable